In [2]:
import pandas as pd
#data = pd.read_csv('training_data_badval.csv')  # just two bad valued data make the F1 scocre from 0.4 to 0.39
data = pd.read_csv('training_data.csv')
data.drop(['Formation','Well Name','Depth'],axis=1)
test_well = data[data['Well Name'] == 'SHANKLE']

data = data[data['Well Name'] != 'SHANKLE']   # including Shankle in training data, give F1 0.57 for test well. 
#data.describe()
#data.hist("Facies")

from sklearn.decomposition import PCA
pca = PCA(n_components = 4)
X2D = pca.fit_transform(scaled_features)
pca.explained_variance_ratio_

ValueError: could not convert string to float: 'CHURCHMAN BIBLE'

In [2]:
data.cov()

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
Facies,6.326478,123.278420,-28.913416,0.222402,-3.564540,-6.590333,1.586454,1.061778,0.047650
Depth,123.278420,19140.551668,-294.232282,6.243325,-63.137710,-91.802830,38.301025,21.339824,0.483657
GR,-28.913416,-294.232282,999.245751,-1.321928,34.034381,63.108791,-9.152689,-4.640281,-1.532588
ILD_log10,0.222402,6.243325,-1.321928,0.058526,-0.170673,-0.942928,0.075355,0.061276,0.007121
DeltaPHI,-3.564540,-63.137710,34.034381,-0.170673,25.461268,-6.672782,-0.192683,-0.587384,0.027833
PHIND,-6.590333,-91.802830,63.108791,-0.942928,-6.672782,54.607145,-3.604716,-1.781553,-0.055986
PE,1.586454,38.301025,-9.152689,0.075355,-0.192683,-3.604716,0.799447,0.288903,0.003619
NM_M,1.061778,21.339824,-4.640281,0.061276,-0.587384,-1.781553,0.288903,0.249519,0.005684
RELPOS,0.047650,0.483657,-1.532588,0.007121,0.027833,-0.055986,0.003619,0.005684,0.082656


In [5]:
#features = ['Depth','GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']  adding depth doesn't improve
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
#features = ['ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS'] #removal of GR make the F1 scocre from 0.4 to 0.38

feature_vectors = data[features]
facies_labels = data['Facies']
facies_labels.describe()

count    2783.000000
mean        4.558390
std         2.515249
min         1.000000
25%         2.000000
50%         4.000000
75%         7.000000
max         9.000000
Name: Facies, dtype: float64

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors) #ndarray now. 
data_s = pd.DataFrame(data=scaled_features)
data_s.describe()
data_s.cov()
#data_s.hist('1')

,0,1,2,3,4,5,6
0,1.000359,-0.172923,0.213451,0.270262,-0.323947,-0.293977,-0.168698
1,-0.172923,1.000359,-0.139864,-0.527637,0.348496,0.507246,0.102414
2,0.213451,-0.139864,1.000359,-0.179019,-0.042723,-0.233124,0.019193
3,0.270262,-0.527637,-0.179019,1.000359,-0.545767,-0.482813,-0.026362
4,-0.323947,0.348496,-0.042723,-0.545767,1.000359,0.647086,0.014085
5,-0.293977,0.507246,-0.233124,-0.482813,0.647086,1.000359,0.039597
6,-0.168698,0.102414,0.019193,-0.026362,0.014085,0.039597,1.000359


array([0.39006   , 0.17100289, 0.14965   , 0.11478091])

In [16]:
from sklearn.cross_validation import train_test_split
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#X_train, X_cv, y_train, y_cv = train_test_split(feature_vectors, facies_labels,test_size=0.05, random_state=42) #not scaling input gives same F1 score. 
X_train, X_cv, y_train, y_cv = train_test_split(X2D, facies_labels,test_size=0.05, random_state=42)



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [18]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=4,min_samples_leaf=10)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [19]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.89      0.50      0.64        16
       CSiS       0.57      0.83      0.68        29
       FSiS       0.31      0.29      0.30        14
       SiSh       0.43      0.43      0.43         7
         MS       0.00      0.00      0.00        18
         WS       0.45      0.87      0.60        23
          D       0.00      0.00      0.00         3
         PS       0.56      0.39      0.46        23
         BS       0.43      0.43      0.43         7

avg / total       0.46      0.51      0.46       140



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
y_test = test_well['Facies']

In [22]:
well_features = test_well.drop(['Facies','Formation','Well Name','Depth'],axis=1)
#well_features = test_well.drop(['Facies','Formation','Well Name'],axis=1)
#well_features = test_well.drop(['Facies','Formation','Well Name','Depth','GR'],axis=1)

X_test = scaler.transform(well_features)
#y_pred = clf.predict(X_test)

pca_test = PCA(n_components = 4)
X2D_test = pca_test.fit_transform(scaled_features)
y_pred = clf.predict(X2D_test)

#y_pred = clf.predict(well_features)
test_well['Prediction'] = y_pred

ValueError: Length of values does not match length of index

In [11]:

print(classification_report(y_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.43      0.07      0.12        89
       CSiS       0.31      0.81      0.45        89
       FSiS       0.82      0.36      0.50       117
       SiSh       0.06      0.14      0.09         7
         MS       0.00      0.00      0.00        19
         WS       0.56      0.80      0.66        71
          D       1.00      0.24      0.38        17
         PS       0.52      0.40      0.45        40

avg / total       0.53      0.44      0.40       449



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
